In [6]:
from PIL import Image
import numpy as np

img = Image.open('Kropka.jpg').convert('L')

img = np.asarray(img)


In [29]:
class Pixel:
    def __init__(self):
        self.maximum = int(0)
        self.normalized = np.float64(0)
        self.raw = np.float64(0)


def picture2sinogram(picture, **kwargs):
    options = {
        'width': 90,
        'detector_amount': 180,
        'alpha': 4
    }
    options.update(kwargs)

    # szerokość rozstawu detektorów
    width = options['width']

    # ilość detektorów
    detector_amount = options['detector_amount']

    # kąt obrotu
    alpha = options['alpha']

    picture_size = len(picture[0])
    r = int(np.ceil(np.sqrt(picture_size * picture_size)))

    sinogram = []
    lines = []

    # poruszaj emiterem 360/n razy o kąt alpha i zbierz próbki promieni.
    for i in range(0, 360, alpha):
        sinogram.append([])
        lines.append([])
        for detector in range(0, detector_amount):
            x0 = r * np.cos(i * np.pi / 180)
            y0 = r * np.sin(i * np.pi / 180)

            x1 = r * np.cos((i + 180 - (width / 2) + detector * (width / (detector_amount - 1))) * np.pi / 180)
            y1 = r * np.sin((i + 180 - (width / 2) + detector * (width / (detector_amount - 1))) * np.pi / 180)

            x0 = int(x0) + np.floor(picture_size / 2)
            x1 = int(x1) + np.floor(picture_size / 2)
            y0 = int(y0) + np.floor(picture_size / 2)
            y1 = int(y1) + np.floor(picture_size / 2)

            line = bresenhams_line(x0, y0, x1, y1)

            pixel = get_pixel_value(picture, line)


            sinogram[-1].append(pixel.normalized)
            lines[-1].append([x0, y0, x1, y1])

    return sinogram, lines


def bresenhams_line(x1, y1, x2, y2):
    ''' Bresenhams algorithm as described here:
    http://eduinf.waw.pl/inf/utils/002_roz/2008_06.php'''

    line = []
    kx = 1 if x1 <= x2 else -1
    ky = 1 if y1 <= y2 else -1

    dx = abs(x2 - x1)
    dy = abs(y2 - y1)

    x = x1
    y = y1
    line.append([x, y])
    if (dx >= dy):
        e = dx / 2
        for i in range(0, int(dx)):
            x = x + kx
            e = e - dy
            if (e < 0):
                y = y + ky
                e = e + dx
            line.append([x, y])
    else:
        e = dy / 2
        for i in range(0, int(dy)):
            y = y + ky
            e = e - dx
            if (e < 0):
                x = x + kx
                e = e + dy
            line.append([x, y])
    return line


def get_pixel_value(picture, line):
    pixel = Pixel()
    for pos in line:
        if pos[0] >= 0 and pos[1] >= 0 and pos[0] < len(picture) and pos[1] < len(picture):
            pixel.raw += float(picture[int(pos[0]),int(pos[1])])
            pixel.maximum += 1
    assert pixel.maximum != 0
    pixel.normalized = pixel.raw / pixel.maximum
    return pixel

In [31]:
detectors_ = 180
iterations_ = 90
detectorsAngles_ = 180 


sinogram, lines = picture2sinogram(img)
sinogram = np.array(sinogram)
print(sinogram.shape)
image2 = Image.fromarray(sinogram)
image2.show()

(90, 180)
